In [1]:
import nipype

from nipype.interfaces import fsl
from nipype.interfaces import ants
from nipype.interfaces import freesurfer
import os

In [2]:
base_dir=os.path.abspath('examples/')
subject_list =sorted(next(os.walk(base_dir))[1])

In [3]:
CORES=4
TASKS=2

In [4]:
grabber = nipype.Node(interface=nipype.DataGrabber(infields=['arg'],outfields=['out_file']), name='grabber')      
grabber.inputs.base_directory = base_dir
grabber.inputs.sort_filelist = True
grabber.inputs.template = '*/%s.nii.gz'
grabber.inputs.arg = 't1w'

In [5]:
sink = nipype.Node(interface=nipype.DataSink(),name='sink')
sink.inputs.base_directory = base_dir
substitutions=[]
for i in range(len(subject_list)):
    substitutions+= [("_neck_remove"+str(i), subject_list[i])]
substitutions+= [("subject_id_", "")]
    
sink.inputs.substitutions =substitutions 

In [6]:
sink2 = nipype.Node(interface=nipype.DataSink(),name='sink2')
sink2.inputs.base_directory = base_dir
substitutions=[]
for i in range(len(subject_list)):
    substitutions+= [("_N4_FC"+str(i), subject_list[i])]
substitutions+= [("subject_id_", "")]
    
sink2.inputs.substitutions =substitutions 

In [7]:
# Neck removal by FSL robustfov
neck_remove=nipype.MapNode(interface=fsl.RobustFOV(), name='neck_remove', iterfield=['in_file'])
neck_remove.inputs.out_roi="t1w_fov.nii.gz"

In [8]:
# Field Inhomogenity estimation (if any) and removal by ANTs N4BiasFieldCorrection
N4_FC=nipype.MapNode(interface=ants.N4BiasFieldCorrection(), name="N4_FC", iterfield=['input_image'])
N4_FC.inputs.dimension = 3
N4_FC.inputs.output_image="t1w_fov_N4.nii.gz"
N4_FC.inputs.num_threads=CORES

In [9]:
workflow = nipype.Workflow('workflow')
workflow.connect([(grabber, neck_remove, [('out_file', 'in_file')]),
                  (neck_remove, sink, [('out_roi', '@in_file')]),
                  (neck_remove, N4_FC, [('out_roi', 'input_image')]),
                  (N4_FC, sink2, [('output_image', '@in_file')])
                 ])

In [10]:
#workflow.run()
workflow.run('MultiProc', plugin_args={'n_procs': TASKS})

170927-19:18:21,616 workflow INFO:
	 Workflow workflow settings: ['check', 'execution', 'logging']
170927-19:18:21,676 workflow INFO:
	 Running in parallel.
170927-19:18:21,680 workflow INFO:
	 Executing: grabber ID: 0
170927-19:18:21,686 workflow INFO:
	 Executing node grabber in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmpYONPEl/workflow/grabber
170927-19:18:21,753 workflow INFO:
	 [Job finished] jobname: grabber jobid: 0
170927-19:18:21,757 workflow INFO:
	 Executing: neck_remove ID: 1
170927-19:18:21,764 workflow INFO:
	 Adding 3 jobs for mapnode neck_remove
170927-19:18:21,769 workflow INFO:
	 Executing: _neck_remove0 ID: 5
170927-19:18:21,772 workflow INFO:
	 Executing: _neck_remove1 ID: 6
170927-19:18:21,777 workflow INFO:
	 Executing node _neck_remove1 in dir: /var/folders/5_/33p0sl3j2_ngqn_gghyy2zj00000gn/T/tmpoXJ6PQ/workflow/neck_remove/mapflow/_neck_remove1
170927-19:18:21,777 workflow INFO:
	 Executing node _neck_remove0 in dir: /var/folders/5_/33p0sl3j2_ngqn_g